### Test the Model (Competition)

In [3]:
%pip install torch
%pip install transformers
%pip install scikit-learn
%pip install pandas
%pip install numpy
%pip install tqdm
%pip install joblib

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import get_scheduler
from torch.optim import AdamW
import torch.nn.functional as F
from tqdm import tqdm
import joblib


In [8]:
df = pd.read_csv('AA_movie_train_data.csv')

In [ ]:
#movie_genre = list(df['Genre'].unique())
#movie_genre.sort()
#movie_genre

['action', 'comedy', 'documentary', 'drama', 'thriller']

In [9]:
df.isna().any()

Title          False
Genre          False
Description    False
dtype: bool

In [10]:
df.shape

(5000, 3)

In [ ]:
# Required Libraries
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm
import torch.nn.functional as F

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. fit on Genre strings
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Genre'])

Using device: cpu


In [ ]:
# 2. train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Description'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# 3. tokenization using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

# 4. create PyTorch Datasets
class MovieDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = MovieDataset(train_encodings, train_labels)
val_dataset = MovieDataset(val_encodings, val_labels)


# 6️. Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)


# 7. training
for epoch in range(num_epochs):
  # 5. Model initialization
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
    model.to(device)
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        total_loss += loss.item() * preds.size(0)
        total_correct += (preds == batch['labels']).sum().item()
        total_samples += preds.size(0)

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    train_accuracy = total_correct / total_samples
    train_loss = total_loss / total_samples

    # Evaluation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_samples = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            val_loss += loss.item() * preds.size(0)
            val_correct += (preds == batch['labels']).sum().item()
            val_samples += preds.size(0)

    val_accuracy = val_correct / val_samples
    val_loss = val_loss / val_samples

    print(f"Epoch {epoch+1}/{num_epochs}:  "
          f"accuracy: {train_accuracy:.4f} - loss: {train_loss:.4f} - "
          f"val_accuracy: {val_accuracy:.4f} - val_loss: {val_loss:.4f}")

# 9. evaluation
model.eval()
train_preds, train_labels = [], []
with torch.no_grad():
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(batch['labels'].cpu().numpy())

train_acc = accuracy_score(train_labels, train_preds)

# Evaluate on validation set
val_preds, val_labels = [], []
with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(batch['labels'].cpu().numpy())

val_acc = accuracy_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds, average='macro')

# Classification report
target_names = label_encoder.inverse_transform(np.unique(val_labels))

print(f"Train Accuracy: {train_acc}")
print(f"Validation Accuracy: {val_acc:}")
print(f"F1 Score (Macro): {f1}")
print("Classification Report:")
print(classification_report(val_labels, val_preds, target_names=target_names))

In [ ]:
# 10. save model in huggingface format
model.save_pretrained("./bert_genre_classifier")
tokenizer.save_pretrained("./bert_genre_classifier")

('./bert_genre_classifier/tokenizer_config.json',
 './bert_genre_classifier/special_tokens_map.json',
 './bert_genre_classifier/vocab.txt',
 './bert_genre_classifier/added_tokens.json')

In [ ]:
import joblib
joblib.dump(label_encoder, "./bert_genre_classifier/label_encoder.pkl")

['./bert_genre_classifier/label_encoder.pkl']

In [ ]:
#### code template test
from transformers import BertForSequenceClassification, BertTokenizer
import joblib
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder


model = BertForSequenceClassification.from_pretrained("./bert_genre_classifier")
tokenizer = BertTokenizer.from_pretrained("./bert_genre_classifier")

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model.to(device)
model.eval()

# load unseen dataset
COMP_DATASET = "AA_movie_comp_template.csv" # replace with path
new_df = pd.read_csv(COMP_DATASET)


# fit on Genre strings
movie_genre = ['action', 'comedy', 'documentary', 'drama', 'thriller'] #taken from training output
label_encoder = joblib.load("./bert_genre_classifier/label_encoder.pkl")
#label_encoder.classes_ = np.array(movie_genre)
new_df['label'] = label_encoder.transform(new_df['Genre'])

texts = new_df["Description"].tolist()

# tokenize
encodings = tokenizer(texts, truncation=True, padding=True, max_length=256, return_tensors="pt")
encodings = {k: v.to(device) for k, v in encodings.items()}

# predict
with torch.no_grad():
    outputs = model(**encodings)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

# Move predictions to CPU and convert to numpy before inverse_transform
predicted_genres = label_encoder.inverse_transform(predictions.cpu().numpy())

# save predictions to new CSV
new_df["predicted_genre"] = predicted_genres
new_df.to_csv("predicted_genres.csv", index=False)

# evaluate
true_labels = np.array(new_df['label'].tolist()) # Convert true_labels to a NumPy array
accuracy = accuracy_score(true_labels, predictions.cpu().numpy())
f1 = f1_score(true_labels, predictions.cpu().numpy(), average='macro')
report = classification_report(true_labels, predictions.cpu().numpy(), target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy:}")
print(f"F1 Score (macro): {f1:}")
print("Classification Report:\n", report)

Using device: cuda
Accuracy: 1.0
F1 Score (macro): 1.0
Classification Report:
               precision    recall  f1-score   support

      action       1.00      1.00      1.00         1
      comedy       1.00      1.00      1.00         1
 documentary       1.00      1.00      1.00         1
       drama       1.00      1.00      1.00         1
    thriller       1.00      1.00      1.00         1

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## Hyperparam Tuning

### 1. Early stopping

In [ ]:
# 2. train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Description'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# 3. tokenization using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

# 4. create PyTorch Datasets
class MovieDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = MovieDataset(train_encodings, train_labels)
val_dataset = MovieDataset(val_encodings, val_labels)

# 5. Model initialization
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
model.to(device)

# 6️. Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 10
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# early stopping
best_val_loss = float('inf')
patience = 2
counter = 0

# 7. training
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        total_loss += loss.item() * preds.size(0)
        total_correct += (preds == batch['labels']).sum().item()
        total_samples += preds.size(0)

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    train_accuracy = total_correct / total_samples
    train_loss = total_loss / total_samples

    # Evaluation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_samples = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            val_loss += loss.item() * preds.size(0)
            val_correct += (preds == batch['labels']).sum().item()
            val_samples += preds.size(0)

    val_accuracy = val_correct / val_samples
    val_loss = val_loss / val_samples

    print(f"Epoch {epoch+1}/{num_epochs}:  "
          f"accuracy: {train_accuracy:.4f} - loss: {train_loss:.4f} - "
          f"val_accuracy: {val_accuracy:.4f} - val_loss: {val_loss:.4f}")

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "best_model.pt")  # Optional: save best model
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

# 9. evaluation
model.eval()
train_preds, train_labels = [], []
with torch.no_grad():
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(batch['labels'].cpu().numpy())

train_acc = accuracy_score(train_labels, train_preds)

# Evaluate on validation set
val_preds, val_labels = [], []
with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(batch['labels'].cpu().numpy())

val_acc = accuracy_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds, average='macro')

# Classification report
target_names = label_encoder.inverse_transform(np.unique(val_labels))

print(f"Train Accuracy: {train_acc}")
print(f"Validation Accuracy: {val_acc:}")
print(f"F1 Score (Macro): {f1}")
print("Classification Report:")
print(classification_report(val_labels, val_preds, target_names=target_names))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/10:   0%|          | 0/250 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Epoch 1/10: 100%|██████████| 250/250 [02:45<00:00,  1.51it/s]


Epoch 1/10:  accuracy: 0.6332 - loss: 0.9477 - val_accuracy: 0.7770 - val_loss: 0.6323


Epoch 2/10: 100%|██████████| 250/250 [02:45<00:00,  1.51it/s]


Epoch 2/10:  accuracy: 0.8367 - loss: 0.4824 - val_accuracy: 0.7510 - val_loss: 0.7088


Epoch 3/10: 100%|██████████| 250/250 [02:45<00:00,  1.51it/s]


Epoch 3/10:  accuracy: 0.9307 - loss: 0.2223 - val_accuracy: 0.7750 - val_loss: 0.7433
Early stopping triggered at epoch 3


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Train Accuracy: 0.984
Validation Accuracy: 0.775
F1 Score (Macro): 0.7760152891260075
Classification Report:
              precision    recall  f1-score   support

      action       0.82      0.75      0.78       200
      comedy       0.80      0.69      0.74       200
 documentary       0.93      0.87      0.90       200
       drama       0.66      0.72      0.69       200
    thriller       0.71      0.85      0.77       200

    accuracy                           0.78      1000
   macro avg       0.78      0.78      0.78      1000
weighted avg       0.78      0.78      0.78      1000



### 2. comparison with flat learning rate

In [ ]:
# 2. train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Description'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# 3. tokenization using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

# 4. create PyTorch Datasets
class MovieDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = MovieDataset(train_encodings, train_labels)
val_dataset = MovieDataset(val_encodings, val_labels)

# 5. Model initialization
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
model.to(device)

# 6️. Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 7. training
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        total_loss += loss.item() * preds.size(0)
        total_correct += (preds == batch['labels']).sum().item()
        total_samples += preds.size(0)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    train_accuracy = total_correct / total_samples
    train_loss = total_loss / total_samples

    # Evaluation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_samples = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            val_loss += loss.item() * preds.size(0)
            val_correct += (preds == batch['labels']).sum().item()
            val_samples += preds.size(0)

    val_accuracy = val_correct / val_samples
    val_loss = val_loss / val_samples

    print(f"Epoch {epoch+1}/{num_epochs}:  "
          f"accuracy: {train_accuracy:.4f} - loss: {train_loss:.4f} - "
          f"val_accuracy: {val_accuracy:.4f} - val_loss: {val_loss:.4f}")

# 9. evaluation
model.eval()
train_preds, train_labels = [], []
with torch.no_grad():
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(batch['labels'].cpu().numpy())

train_acc = accuracy_score(train_labels, train_preds)

# Evaluate on validation set
val_preds, val_labels = [], []
with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(batch['labels'].cpu().numpy())

val_acc = accuracy_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds, average='macro')

# Classification report
target_names = label_encoder.inverse_transform(np.unique(val_labels))

print(f"== FLAT LEARNING RATE ==")
print(f"Train Accuracy: {train_acc}")
print(f"Validation Accuracy: {val_acc:}")
print(f"F1 Score (Macro): {f1}")
print("Classification Report:")
print(classification_report(val_labels, val_preds, target_names=target_names))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3:   0%|          | 0/250 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Epoch 1/3: 100%|██████████| 250/250 [02:45<00:00,  1.51it/s]


Epoch 1/3:  accuracy: 0.6635 - loss: 0.9120 - val_accuracy: 0.7250 - val_loss: 0.7479


Epoch 2/3: 100%|██████████| 250/250 [02:45<00:00,  1.51it/s]


Epoch 2/3:  accuracy: 0.8385 - loss: 0.4752 - val_accuracy: 0.7560 - val_loss: 0.7341


Epoch 3/3: 100%|██████████| 250/250 [02:45<00:00,  1.51it/s]


Epoch 3/3:  accuracy: 0.9230 - loss: 0.2344 - val_accuracy: 0.7390 - val_loss: 0.8958


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


== FLAT LEARNING RATE ==
Train Accuracy: 0.97075
Validation Accuracy: 0.739
F1 Score (Macro): 0.7384879352049007
Classification Report:
              precision    recall  f1-score   support

      action       0.81      0.70      0.75       200
      comedy       0.87      0.58      0.69       200
 documentary       0.86      0.90      0.88       200
       drama       0.62      0.68      0.65       200
    thriller       0.63      0.84      0.72       200

    accuracy                           0.74      1000
   macro avg       0.76      0.74      0.74      1000
weighted avg       0.76      0.74      0.74      1000



### 3. automated tuning w optuna

In [10]:
!pip install optuna
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


In [14]:
import optuna

from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
import torch

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Description'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# 3. tokenization using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

# 4. create PyTorch Datasets
class MovieDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = MovieDataset(train_encodings, train_labels)
val_dataset = MovieDataset(val_encodings, val_labels)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Move tensors to CPU before converting to numpy
    predictions = np.argmax(logits, axis=-1)
    labels_cpu = labels
    f1 = f1_score(labels_cpu, predictions, average="macro")
    accuracy = accuracy_score(labels_cpu, predictions)
    return {"eval_f1": f1, "eval_accuracy": accuracy}


def objective(trial):
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
    model.to(device)
    learning_rate = trial.suggest_float("learning_rate", 3e-5, 8e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32])
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 5)

    training_args = TrainingArguments(
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        output_dir="./results",
        logging_dir="./logs",
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="epoch",
        metric_for_best_model="eval_f1",
        greater_is_better=True,
        report_to="none", # Disable reporting to wandb
        save_total_limit=1,
        fp16=torch.cuda.is_available()
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train() # Train the model

    result = trainer.evaluate()
    return result["eval_f1"]

study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=0))
study.optimize(objective, n_trials=10)

print(study.best_params)

[I 2025-08-05 14:07:02,786] A new study created in memory with name: no-name-342fbe9e-f712-4822-b502-d70bab3c518c
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.763200,0.679775,0.750034,0.750000
2,0.395900,0.691220,0.755082,0.757000
3,0.161800,0.777364,0.773098,0.773000


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[I 2025-08-05 14:11:00,805] Trial 0 finished with value: 0.773097765908797 and parameters: {'learning_rate': 4.288736962320381e-05, 'batch_size': 16, 'num_train_epochs': 3}. Best is trial 0 with value: 0.773097765908797.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.789900,0.728059,0.752236,0.753000
2,0.381100,0.700401,0.763067,0.764000
3,0.134100,0.880800,0.788629,0.789000


...

[I 2025-08-05 14:49:11,228] Trial 8 finished with value: 0.7806278654793348 and parameters: {'learning_rate': 4.405721962094015e-05, 'batch_size': 32, 'num_train_epochs': 3}. Best is trial 1 with value: 0.7886292113867858.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.960500,0.688836,0.749904,0.755000
2,0.517900,0.647131,0.770906,0.771000
3,0.311500,0.755673,0.783683,0.784000
4,0.062500,0.938316,0.772352,0.773000
5,0.024000,1.012746,0.772296,0.771000


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packa

[I 2025-08-05 14:54:52,262] Trial 9 finished with value: 0.7722959280626938 and parameters: {'learning_rate': 6.4198623939046e-05, 'batch_size': 32, 'num_train_epochs': 5}. Best is trial 1 with value: 0.7886292113867858.


{'learning_rate': 7.722228565978238e-05, 'batch_size': 16, 'num_train_epochs': 3}


best params from optuna

In [20]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score, classification_report
import numpy as np
import torch

# Get the best hyperparameters from the Optuna study
best_learning_rate = study.best_params['learning_rate']
best_batch_size = study.best_params['batch_size']
best_num_train_epochs = study.best_params['num_train_epochs']

print(f"Best Learning Rate: {best_learning_rate}")
print(f"Best Batch Size: {best_batch_size}")
print(f"Best Number of Training Epochs: {best_num_train_epochs}")

# 5. Model initialization (re-initialize for training with best params)
# Make sure to load the model again if you want to train from scratch
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Define compute_metrics function for Trainer
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average="macro")
    accuracy = accuracy_score(labels, predictions)
    return {"eval_f1": f1, "eval_accuracy": accuracy}

# 6. Training Arguments with best hyperparameters and fp16 enabled
training_args = TrainingArguments(
    output_dir="./results_best_params",
    learning_rate=best_learning_rate,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size, # Use best batch size for evaluation as well
    num_train_epochs=best_num_train_epochs,
    weight_decay=0.01, # Added a small weight decay for regularization
    eval_strategy="epoch",
    save_strategy="epoch",
    # Removed load_best_model_at_last_k_epochs
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(), # Enable fp16 if CUDA is available
    report_to="none" # Disable reporting to wandb
)

# 7. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# 8. Train the model
# Modify the train function to print the learning rate at the end of each epoch
original_train = trainer.train
def train_with_lr_print(*args, **kwargs):
    original_train(*args, **kwargs)
    for epoch in range(int(trainer.state.epoch)):
        print(f"Epoch {epoch+1} learning rate: {trainer.lr_scheduler.get_last_lr()[0]}")

trainer.train = train_with_lr_print

trainer.train()

# 9. Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

# Optional: Save the best model
trainer.save_model("./best_model_best_params")

Best Learning Rate: 7.722228565978238e-05
Best Batch Size: 16
Best Number of Training Epochs: 3


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,0.706920,0.753133,0.754000
2,0.639900,0.725174,0.755841,0.759000
3,0.639900,0.946137,0.771284,0.770000


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch 1 learning rate: 2.05926095092753e-07
Epoch 2 learning rate: 2.05926095092753e-07
Epoch 3 learning rate: 2.05926095092753e-07


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_f1': 0.771284361423979, 'eval_accuracy': 0.77, 'eval_loss': 0.9461365938186646, 'eval_runtime': 3.4648, 'eval_samples_per_second': 288.62, 'eval_steps_per_second': 18.183, 'epoch': 3.0}


In [21]:
# Get predictions and true labels for classification report
train_predictions = trainer.predict(train_dataset).predictions
train_predicted_labels = np.argmax(train_predictions, axis=-1)
train_true_labels = train_dataset.labels

val_predictions = trainer.predict(val_dataset).predictions
val_predicted_labels = np.argmax(val_predictions, axis=-1)
val_true_labels = val_dataset.labels


# Calculate and print train accuracy and F1 score
train_accuracy = accuracy_score(train_true_labels, train_predicted_labels)
train_f1 = f1_score(train_true_labels, train_predicted_labels, average='macro')

print(f"\nTrain Accuracy: {train_accuracy}")

# Calculate and print validation accuracy and F1 score
val_accuracy = accuracy_score(val_true_labels, val_predicted_labels)
val_f1 = f1_score(val_true_labels, val_predicted_labels, average='macro')

print(f"\nValidation Accuracy: {val_accuracy}")
print(f"Validation F1 Score (Macro): {val_f1}")

# Generate and print classification report
target_names = label_encoder.classes_
report = classification_report(val_true_labels, val_predicted_labels, target_names=target_names)
print("\nClassification Report:")
print(report)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Train Accuracy: 0.985

Validation Accuracy: 0.77
Validation F1 Score (Macro): 0.771284361423979

Classification Report:
              precision    recall  f1-score   support

      action       0.78      0.77      0.77       200
      comedy       0.83      0.67      0.74       200
 documentary       0.91      0.90      0.90       200
       drama       0.62      0.71      0.66       200
    thriller       0.75      0.80      0.78       200

    accuracy                           0.77      1000
   macro avg       0.78      0.77      0.77      1000
weighted avg       0.78      0.77      0.77      1000

